In [ ]:
from warnings import filterwarnings
filterwarnings("ignore")

import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pyplot as plt
import numpy as np
import os

# Data loading and pre-processing
from keras.datasets import cifar10
from keras.utils import np_utils

from keras.preprocessing.image import ImageDataGenerator

# Model related imports
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

from keras.optimizers import SGD, RMSprop, Adam

In [ ]:
def create_dir_if_not_exists(directory):
    """Creates a directory if it doesn't already exist.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)

def save_loss_plots(history, directory):
    """Saves a plot of the training and test curves.
    """
    for metric in list(['accuracy','loss']):
        plt.figure()
        plt.plot(history.history[metric])
        plt.plot(history.history['val_'+metric])
        plt.title('Model '+ metric)
        plt.ylabel(metric)
        plt.xlabel('epoch')
        plt.legend(['train','test'], loc='upper left')
        plt.tight_layout()
        plt.savefig(os.path.join(directory, metric+'_plot.pdf'))
        plt.close()

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# convert to float32, so that division does not make the features all zero
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# divide features by 255 so that pixels have value in the range of [0,1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices (one-hot encoding)
n_output = 10
y_train = np_utils.to_categorical(y_train, n_output)
y_test = np_utils.to_categorical(y_test, n_output)

print("X_Train: {}\nX_Test:  {}\nY_Train: {}\nY_Test:  {}"\
        .format(x_train.shape, x_test.shape,y_train.shape, y_test.shape))

In [ ]:
# Creating the model

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=10, activation='softmax'))

optim = SGD(lr=0.001, momentum=0.9, nesterov=False)
#optim = RMSprop(learning_rate=0.001, rho=0.9)
#optim = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

model.compile(loss='categorical_crossentropy',
              optimizer=optim,
              metrics=['accuracy'])

In [ ]:
#for layer in model.get_weights():
#    print(np.average(layer))

In [ ]:
history = model.fit(x_train, y_train, batch_size=100, epochs=2, validation_data=(x_test,y_test))

In [ ]:
BS = 50
train_datagen = ImageDataGenerator(width_shift_range=0.15, height_shift_range=0.15, horizontal_flip=True)

train_generator = train_datagen.flow(x_train, y_train, batch_size=BS)

val_datagen = ImageDataGenerator()
val_generator = val_datagen.flow(x_test, y_test, batch_size=BS)


history = model.fit_generator(train_generator,
                              steps_per_epoch = len(x_train) // BS,
                              epochs=50,
                              validation_data=val_generator,
                              validation_steps= len(x_test) // BS)

In [ ]:
create_dir_if_not_exists('./checkpoints')
save_loss_plots(history, './checkpoints')
model.save_weights(os.path.join('./checkpoints',model_name, 'model.h5'))